# Optimization Techniques


## Contents

1. A/B Testing
2. Iterative Refining
3. Evaluation Metric


## Setup


In [3]:
import re
import numpy as np


from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate


llm = ChatOllama(model="llama3.2")


# Define a helper function to generate responses
def generate_response(prompt):
    """Generate a response using the language model.

    Args:
        prompt (str): The input prompt.

    Returns:
        str: The generated response.
    """
    return llm.invoke(prompt).content

## Evaluation Metric


In [4]:
def evaluate_response(response, criteria):
    """Evaluate the quality of a response based on given criteria.

    Args:
        response (str): The generated response.
        criteria (list): List of criteria to evaluate.

    Returns:
        float: The average score across all criteria.
    """
    scores = []
    for criterion in criteria:
        print(f"Evaluating response based on {criterion}...")
        prompt = f"On a scale of 1-10, rate the following response on {criterion}. Start your response with the numeric score:\n\n{response}"
        response = generate_response(prompt)
        # show 50 characters of the response
        # Use regex to find the first number in the response
        score_match = re.search(r"\d+", response)
        if score_match:
            score = int(score_match.group())
            scores.append(min(score, 10))  # Ensure score is not greater than 10
        else:
            print(
                f"Warning: Could not extract numeric score for {criterion}. Using default score of 5."
            )
            scores.append(5)  # Default score if no number is found
    return np.mean(scores)

## A/B Testing


In [5]:
# Define prompt variations
prompt_a = PromptTemplate(
    input_variables=["topic"], template="Explain {topic} in simple terms."
)

prompt_b = PromptTemplate(
    input_variables=["topic"],
    template="Provide a beginner-friendly explanation of {topic}, including key concepts and an example.",
)

In [ ]:
# Perform A/B test
topic = "machine learning"
response_a = generate_response(prompt_a.format(topic=topic))
response_b = generate_response(prompt_b.format(topic=topic))

In [9]:
criteria = ["clarity", "informativeness", "engagement"]
score_a = evaluate_response(response_a, criteria)
score_b = evaluate_response(response_b, criteria)

print(f"Prompt A score: {score_a:.2f}")
print(f"Prompt B score: {score_b:.2f}")
print(f"Winning prompt: {'A' if score_a > score_b else 'B'}")

Evaluating response based on clarity...
Evaluating response based on informativeness...
Evaluating response based on engagement...
Evaluating response based on clarity...
Evaluating response based on informativeness...
Evaluating response based on engagement...
Prompt A score: 7.00
Prompt B score: 8.00
Winning prompt: B


## Iterative Refinement


In [16]:
def refine_prompt(initial_prompt, topic, iterations=3):
    """Refine a prompt through multiple iterations.

    Args:
        initial_prompt (PromptTemplate): The starting prompt template.
        topic (str): The topic to explain.
        iterations (int): Number of refinement iterations.

    Returns:
        PromptTemplate: The final refined prompt template.
    """
    current_prompt = initial_prompt
    for i in range(iterations):
        try:
            response = generate_response(current_prompt.format(topic=topic))
        except KeyError as e:
            print(f"Error in iteration {i+1}: Missing key {e}. Adjusting prompt...")
            # Remove the problematic placeholder
            current_prompt.template = current_prompt.template.replace(
                f"{{{e.args[0]}}}", "relevant example"
            )
            response = generate_response(current_prompt.format(topic=topic))

        # Generate feedback and suggestions for improvement
        feedback_prompt = f"Analyze the following explanation of {topic} and suggest improvements to the prompt that generated it:\n\n{response}"
        feedback = generate_response(feedback_prompt)

        # Use the feedback to refine the prompt
        refine_prompt = f"Based on this feedback: '{feedback}', improve the following prompt template. Ensure to only use the variable {{topic}} in your template:\n\n{current_prompt.template}"
        refined_template = generate_response(refine_prompt)

        current_prompt = PromptTemplate(
            input_variables=["topic"], template=refined_template
        )

        print(f"Iteration {i+1} prompt: {current_prompt.template}")

    return current_prompt


# Perform A/B test
topic = "machine learning"
response_a = generate_response(prompt_a.format(topic=topic))
response_b = generate_response(prompt_b.format(topic=topic))

criteria = ["clarity", "informativeness", "engagement"]
score_a = evaluate_response(response_a, criteria)
score_b = evaluate_response(response_b, criteria)

print(f"Prompt A score: {score_a:.2f}")
print(f"Prompt B score: {score_b:.2f}")
print(f"Winning prompt: {'A' if score_a > score_b else 'B'}")

# Start with the winning prompt from A/B testing
initial_prompt = prompt_b if score_b > score_a else prompt_a
refined_prompt = refine_prompt(initial_prompt, "machine learning")

print("\nFinal refined prompt:")
print(refined_prompt.template)

Evaluating response based on clarity...
Evaluating response based on informativeness...
Evaluating response based on engagement...
Evaluating response based on clarity...
Evaluating response based on informativeness...
Evaluating response based on engagement...
Prompt A score: 8.00
Prompt B score: 8.00
Winning prompt: B
Iteration 1 prompt: Here is an updated version of the prompt template incorporating the provided feedback suggestions:

**Template:**

Imagine a world where computers can learn from data without being explicitly programmed. Welcome to the realm of {topic}!

In traditional programming, developers tell computers exactly what to do, step by step. But in machine learning, we teach computers to figure things out on their own using examples and patterns in data.

**A Simplified Example:**

Let's say you want to train a computer to recognize objects or solve a problem. You start by showing it many instances of the task, like images of animals, customer interactions, or financi

KeyError: 'Domain 1'

## Identifying Ambiguous Prompts

In [15]:
ambiguous_prompts = [
    "Tell me about the bank.",
    "What's the best way to get to school?",
    "Can you explain the theory?"
]

for prompt in ambiguous_prompts:
    analysis_prompt = f"Analyze the following prompt for ambiguity: '{prompt}'. Explain why it's ambiguous and list possible interpretations."
    print(f"Prompt: {prompt}")
    print(llm.invoke(analysis_prompt).content)
    print("-" * 50)

Prompt: Tell me about the bank.
The prompt "Tell me about the bank" is ambiguous because it lacks specificity, allowing for multiple possible interpretations. Here are some reasons why:

1. **Lack of clarity on what kind of bank**: The phrase "the bank" could refer to a variety of institutions, such as a financial institution, a commercial bank, an online banking platform, or even a specific type of bank like a savings bank or investment bank.
2. **Possible meanings for the word "bank"**: In addition to referring to a financial institution, "bank" can also mean a slope or incline, especially in geography (e.g., "the river bank") or architecture (e.g., "a bank of buildings").
3. **Ambiguity about scope**: The prompt doesn't specify what aspect of the bank should be discussed. Should it be the history, services offered, products, management, policies, or something else?
4. **Multiple possible interpretations**: Considering these factors, possible interpretations of the prompt could inclu

## Resolving Ambiguity

In [17]:
def resolve_ambiguity(prompt, context):
    """
    Resolve ambiguity in a prompt by providing additional context.
    
    Args:
    prompt (str): The original ambiguous prompt
    context (str): Additional context to resolve ambiguity
    
    Returns:
    str: The AI's response to the clarified prompt
    """
    clarified_prompt = f"{context}\n\nBased on this context, {prompt}"
    return llm.invoke(clarified_prompt).content

# Example usage
ambiguous_prompt = "Tell me about the bank."
contexts = [
    "You are a financial advisor discussing savings accounts.",
    "You are a geographer describing river formations."
]

for context in contexts:
    print(f"Context: {context}")
    print(f"Clarified response: {resolve_ambiguity(ambiguous_prompt, context)}")
    print("-" * 50)

Context: You are a financial advisor discussing savings accounts.
Clarified response: As a financial advisor, I'm happy to share with you the features and benefits of our institution's savings account.

Our bank is a well-established, reputable financial institution that offers a range of savings options to suit various needs and goals. Our savings accounts are designed to help you build wealth over time, while also providing easy access to your funds when needed.

Here are some key features of our savings account:

1. **High-Yield Interest Rate**: We offer a competitive interest rate on our savings accounts, which can help you earn more interest on your deposits.
2. **Low Fees**: Our fees are minimal and transparent, so you can keep more of your hard-earned money in your account.
3. **Liquidity**: Our savings accounts are liquid, meaning you can access your funds whenever you need them.
4. **Security**: Your deposits are insured by the FDIC (Federal Deposit Insurance Corporation), whi

### Techniques for Writing Clearer Prompts


In [18]:
def compare_prompt_clarity(original_prompt, improved_prompt):
    """
    Compare the responses to an original prompt and an improved, clearer version.
    
    Args:
    original_prompt (str): The original, potentially unclear prompt
    improved_prompt (str): An improved, clearer version of the prompt
    
    Returns:
    tuple: Responses to the original and improved prompts
    """
    original_response = llm.invoke(original_prompt).content
    improved_response = llm.invoke(improved_prompt).content
    return original_response, improved_response

# Example usage
original_prompt = "How do I make it?"
improved_prompt = "Provide a step-by-step guide for making a classic margherita pizza, including ingredients and cooking instructions."

original_response, improved_response = compare_prompt_clarity(original_prompt, improved_prompt)

print("Original Prompt Response:")
print(original_response)
print("\nImproved Prompt Response:")
print(improved_response)

Original Prompt Response:
I'm happy to help, but I need a bit more information. You didn't specify what you're trying to make. Could you please provide more context or clarify what you're looking to create (e.g., a recipe, a craft project, a DIY item, etc.)? I'll do my best to assist you.

Improved Prompt Response:
Here's a step-by-step guide to making a classic Margherita pizza:

Ingredients:

* 1 1/2 cups warm water
* 1 tablespoon active dry yeast
* 3 tablespoons olive oil
* 1 teaspoon salt
* 4 cups all-purpose flour (preferably "00" or Caputo flour)
* 1 cup San Marzano tomatoes, crushed by hand
* 8 ounces fresh mozzarella cheese, sliced into thin rounds
* Fresh basil leaves
* Extra-virgin olive oil for brushing the crust

Cooking Instructions:

**Step 1: Make the Pizza Dough (about 1 hour and 30 minutes)**

1. In a large mixing bowl, combine warm water and yeast. Let it sit for 5-10 minutes until the yeast becomes frothy.
2. Add olive oil, salt, and 2 cups of flour to the bowl. Mix 

### Structured Prompts for Clarity


In [19]:
structured_prompt = PromptTemplate(
    input_variables=["topic", "aspects", "tone"],
    template="""Provide an analysis of {topic} considering the following aspects:
    1. {{aspects[0]}}
    2. {{aspects[1]}}
    3. {{aspects[2]}}
    
    Present the analysis in a {tone} tone.
    """
)

# Example usage
input_variables = {
    "topic": "the impact of social media on society",
    "aspects": ["communication patterns", "mental health", "information spread"],
    "tone": "balanced and objective"
}

chain = structured_prompt | llm
response = chain.invoke(input_variables).content
print(response)

Based on the provided aspects, here is an analysis of the impact of social media on society:

1. **Mental Health and Well-being:**

Social media has been shown to have both positive and negative effects on mental health and well-being. On the one hand, social media platforms can provide a sense of community and connection for people who may be isolated or struggling with loneliness. Social support groups, online forums, and social media communities can offer emotional support, advice, and a sense of belonging. Additionally, social media can also be used to promote mental health awareness, reduce stigma around mental illness, and provide access to mental health resources.

On the other hand, excessive social media use has been linked to increased symptoms of depression, anxiety, and loneliness. The constant stream of curated and often unrealistic content can create unrealistic expectations and promote a culture of comparison, where individuals feel inadequate or insufficient compared to

In [20]:
unclear_prompts = [
    "What's the difference?",
    "How does it work?",
    "Why is it important?"
]

def improve_prompt_clarity(unclear_prompt):
    """
    Improve the clarity of a given prompt.
    
    Args:
    unclear_prompt (str): The original unclear prompt
    
    Returns:
    str: An improved, clearer version of the prompt
    """
    improvement_prompt = f"The following prompt is unclear: '{unclear_prompt}'. Please provide a clearer, more specific version of this prompt. output just the improved prompt and nothing else." 
    return llm.invoke(improvement_prompt).content

for prompt in unclear_prompts:
    improved_prompt = improve_prompt_clarity(prompt)
    print(f"Original: {prompt}")
    print(f"Improved: {improved_prompt}")
    print("-" * 50)

Original: What's the difference?
Improved: What are the key differences between two or more concepts, ideas, or things that you would like me to compare?
--------------------------------------------------
Original: How does it work?
Improved: What are the underlying mechanics or principles behind this system/component/technology?
--------------------------------------------------
Original: Why is it important?
Improved: What are the potential consequences or benefits of [specific topic or action]?
--------------------------------------------------


# Managing Prompt Length and Complexity

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain

In [22]:
# Detailed prompt
detailed_prompt = PromptTemplate(
    input_variables=["topic"],
    template="""Please provide a comprehensive explanation of {topic}. Include its definition, 
    historical context, key components, practical applications, and any relevant examples. 
    Also, discuss any controversies or debates surrounding the topic, and mention potential 
    future developments or trends."""
)

# Concise prompt
concise_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Briefly explain {topic} and its main importance."
)

topic = "artificial intelligence"

print("Detailed response:")
print(llm.invoke(detailed_prompt.format(topic=topic)).content)

print("\nConcise response:")
print(llm.invoke(concise_prompt.format(topic=topic)).content)

Detailed response:
Artificial intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, decision-making, and perception. AI involves the creation of algorithms and statistical models that enable machines to analyze data, recognize patterns, and make predictions or decisions.

Historical Context:

The concept of AI dates back to ancient Greece, where myths told of artificial beings created by the gods. However, the modern concept of AI emerged in the mid-20th century with the work of Alan Turing, who proposed the Turing Test as a measure of a machine's ability to exhibit intelligent behavior equivalent to, or indistinguishable from, that of a human.

In the 1950s and 1960s, AI research focused on rule-based systems and expert systems. The first AI program, called Logical Theorist, was developed in 1956 by Allen Newell and Herbert Simon. However, it wasn't until the 1980s that AI be

## Analysis of Prompt Balance

In [23]:
analysis_prompt = PromptTemplate(
    input_variables=["detailed_response", "concise_response"],
    template="""Compare the following two responses on artificial intelligence:

Detailed response:
{detailed_response}

Concise response:
{concise_response}

Analyze the differences in terms of:
1. Information coverage
2. Clarity and focus
3. Potential use cases for each type of response

Then, suggest strategies for balancing detail and conciseness in prompts."""
)

detailed_response = llm.invoke(detailed_prompt.format(topic=topic)).content
concise_response = llm.invoke(concise_prompt.format(topic=topic)).content

analysis = llm.invoke(analysis_prompt.format(
    detailed_response=detailed_response,
    concise_response=concise_response
)).content

print(analysis)

**Differences Analysis**

1. **Information Coverage**:
	* Detailed Response: Provides a comprehensive overview of AI, including its definition, historical context, key components, practical applications, examples, controversies, and future developments.
	* Concise Response: Focuses on the core aspects of AI, such as automation, decision-making, customer experience, innovation, and efficiency.
2. **Clarity and Focus**:
	* Detailed Response: Explains complex concepts like machine learning, deep learning, and natural language processing in detail, making it suitable for an audience with a technical background or interested in the intricacies of AI.
	* Concise Response: Simplifies these concepts and focuses on their practical applications, making it more accessible to a broader audience.
3. **Potential Use Cases**:
	* Detailed Response: Suitable for educational purposes, research papers, or articles targeting an audience with a technical background or interested in the theoretical aspects 

## Process-Flow for Handling Long Contexts

### 1. Chunking

In [24]:
long_text = """
Artificial intelligence (AI) is a branch of computer science that aims to create intelligent machines that can simulate human cognitive processes.
The field of AI has a rich history dating back to the 1950s, with key milestones such as the development of the first neural networks and expert systems.
AI encompasses a wide range of subfields, including machine learning, natural language processing, computer vision, and robotics.
Practical applications of AI include speech recognition, image classification, autonomous vehicles, and medical diagnosis.
AI has the potential to revolutionize many industries, from healthcare and finance to transportation and entertainment.
However, there are ongoing debates and controversies surrounding AI, such as concerns about job displacement, bias in algorithms, and the ethical implications of autonomous systems.
Looking ahead, the future of AI holds promise for advancements in areas like explainable AI, AI ethics, and human-AI collaboration. 
The intersection of AI with other technologies like blockchain, quantum computing, and biotechnology will likely shape the future of the field.
But as AI continues to evolve, it is essential to consider the societal impact and ethical implications of these technologies.
One of the key challenges for AI researchers and developers is to strike a balance between innovation and responsibility, ensuring that AI benefits society as 
a whole while minimizing potential risks.
If managed effectively, AI has the potential to transform our world in ways we can only begin to imagine.
Though the future of AI is uncertain, one thing is clear: the impact of artificial intelligence will be profound and far-reaching.
"""

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

# Split the text into chunks
chunks = text_splitter.split_text(long_text)

print(f"Number of chunks: {len(chunks)}")
print(f"First chunk: {chunks[0][:200]}...")

Number of chunks: 2
First chunk: Artificial intelligence (AI) is a branch of computer science that aims to create intelligent machines that can simulate human cognitive processes.
The field of AI has a rich history dating back to the...


### 2. Summarization

In [26]:
from langchain.docstore.document import Document

# Convert text chunks to Document objects
doc_chunks = [Document(page_content=chunk) for chunk in chunks]

# Load the summarization chain
chain = load_summarize_chain(llm, chain_type="map_reduce")

# Summarize the long text
summary_result = chain.invoke(doc_chunks)

print("Summary:")
print(summary_result['output_text'])

/home/kalki/genAI/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Summary:
Artificial intelligence (AI) is a branch of computer science aimed at creating intelligent machines, with numerous subfields and practical applications. While it holds promise for revolutionizing industries, it also raises concerns about job displacement, bias, and ethics. The future of AI promises advancements in explainability, ethics, and human-AI collaboration, but requires careful consideration of societal implications to unlock its full potential.


### 3. Iterative Processing

In [27]:
def iterative_analysis(text, steps):
    """
    Perform iterative analysis on a given text.
    
    Args:
    text (str): The text to analyze.
    steps (list): List of analysis steps to perform.
    
    Returns:
    str: The final analysis result.
    """
    result = text
    for step in steps:
        prompt = PromptTemplate(
            input_variables=["text"],
            template=f"Analyze the following text. {step}\n\nText: {{text}}\n\nAnalysis:"
        )
        result = llm.invoke(prompt.format(text=result)).content
    return result

analysis_steps = [
    "Identify the main topics discussed.",
    "Summarize the key points for each topic.",
    "Provide a brief conclusion based on the analysis."
]

final_analysis = iterative_analysis(long_text, analysis_steps)
print("Final Analysis:")
print(final_analysis)

Final Analysis:
Based on the analysis, it appears that the text provides a general overview of Artificial Intelligence (AI) and its various aspects. The summary sections highlight the definition, history, scope, applications, challenges, and future developments of AI. However, the text lacks specific details and concrete examples to support these points.

One notable aspect is the emphasis on the potential benefits and challenges of AI, highlighting its transformative impact on industries and society as a whole. The text also acknowledges the importance of considering ethical implications, societal implications, and responsible development in the field of AI.

The analysis reveals that the text appears to be more of an introduction or overview of AI, rather than a comprehensive treatment of the subject. The lack of specific examples and concrete data suggests that the text may not be intended for an academic or technical audience seeking in-depth knowledge on the topic.

In conclusion,